diff : deleting html better or not ?

In [298]:
CLIP_FEATURE_SIZE=768


In [299]:
!pip install torcheval

In [300]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import copy
import nltk

from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

# import for NLP
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical


from sklearn.metrics import f1_score



In [301]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [302]:
from nltk.corpus import stopwords

In [303]:
final_stopwords_list = stopwords.words('english') + stopwords.words('french')+stopwords.words('german')# Fr,En,De Stopwords

In [304]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [305]:
# mount the drive where your dataset is available
from google.colab import drive
drive.mount('/content/drive')
filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [306]:
# Load data
X_train = pd.read_csv(filepath+'X_train.csv')
y_train = pd.read_csv(filepath+'Y_train.csv')
X_train=X_train.drop(columns="Unnamed: 0")
y_train=y_train.drop(columns="Unnamed: 0")

In [307]:
# Cleaning and Preprocessing Text
CLEANR = re.compile('<.*?>') # delete html tag

def clean_html(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

def clean_text(text):
    text=clean_html(text)
    # text=re.sub('\d', '0', text)#replace all digits by 0
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-ZäöüßÄÖÜ ]', '', text)
    # Convert text to lowercase
    text = text.lower()
    # text=text.split(" ")
    # for word in text:
    #     if word in final_stopwords_list:
    #         text.remove(word)
    # text=" ".join(text)
    return text

In [308]:
# Apply cleaning function to the 'designation' column
X_train['designation'] = X_train['designation'].fillna('').apply(clean_text)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['designation'])
sequences = tokenizer.texts_to_sequences(X_train['designation'])

# Padding to max length of text
data = pad_sequences(sequences, maxlen=34)

# Assuming the number of unique words in the tokenizer plus 1 is vocab_size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)# nearly 70 000 of vocab size, it seems too much

69188


In [309]:
# Split data into training and validation set (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(data, y_train, test_size=0.2,shuffle=False)
y_train=y_train["prdtypecode"].tolist()
y_val=y_val["prdtypecode"].tolist()

In [310]:
# Convert labels to categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Convert to one-hot encoding
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)


In [311]:
class TextDataset(Dataset):
    def __init__(self, X, Y):
        self.inputs = X
        self.labels = Y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = torch.tensor(self.inputs[idx]).to(device)
        y = torch.tensor(self.labels[idx], dtype=torch.float).to(device)
        return x, y

In [312]:
embedding_dim = 300
batch_size=128

In [313]:
train_dataset=TextDataset(X_train,y_train_categorical)
train_loader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset=TextDataset(X_val,y_val_categorical)
val_loader=DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


# Model definition


In [314]:

# Define the model
class CNN_classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super(CNN_classifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_blocks = nn.ModuleList([
            nn.Conv2d(1, 512, (i, embedding_dim), padding=(0, 0))
            for i in range(1,7)
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512 * 6, CLIP_FEATURE_SIZE)
        self.classif=nn.Linear(CLIP_FEATURE_SIZE,num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add channel dimension for Conv2d
        conv_outputs = [nn.functional.relu(conv_block(x)).max(dim=3)[0].max(dim=2)[0] for conv_block in self.conv_blocks]# [0] to get only the values and not the indices ( in pos 1 )
        x = torch.cat(conv_outputs, dim=1)
        # Dense Layer

        # Flatten Layer
        x = x.view(x.size(0), -1)
        # Dropout Layer
        x = self.dropout(x)

        x = self.fc(x)
        x = self.classif(x)
        return x


In [315]:
# Initialize the model
num_classes = 27
model = CNN_classifier(vocab_size, embedding_dim, num_classes)

# Convert the model to CUDA if available
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
# Print the model summary
print(model)


CNN_classifier(
  (embedding): Embedding(69188, 300)
  (conv_blocks): ModuleList(
    (0): Conv2d(1, 512, kernel_size=(1, 300), stride=(1, 1))
    (1): Conv2d(1, 512, kernel_size=(2, 300), stride=(1, 1))
    (2): Conv2d(1, 512, kernel_size=(3, 300), stride=(1, 1))
    (3): Conv2d(1, 512, kernel_size=(4, 300), stride=(1, 1))
    (4): Conv2d(1, 512, kernel_size=(5, 300), stride=(1, 1))
    (5): Conv2d(1, 512, kernel_size=(6, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=3072, out_features=768, bias=True)
  (classif): Linear(in_features=768, out_features=27, bias=True)
)


In [316]:
def train_model(model,train_loader,val_loader,num_epochs=10):  # Train the model
  val_f1=0
  max_val_f1=0
  for epoch in tqdm(range(num_epochs)):
      model.train()
      for inputs, labels in tqdm(train_loader,desc=f"Epoch {epoch + 1}/{num_epochs}, Validation F1 Score: {val_f1:.4f}"):
          optimizer.zero_grad()
          outputs = model(inputs)
          outputs=outputs.squeeze(0)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

      model.eval()
      with torch.no_grad():
          val_preds = []
          val_labels = []
          for val_inputs, val_labels_batch in val_loader:
              val_outputs = model(val_inputs)
              val_preds.append(val_outputs.cpu())
              val_labels.append(val_labels_batch.cpu())

      val_preds = torch.cat(val_preds, dim=0)
      val_labels = torch.cat(val_labels, dim=0)

    #   print(f'{val_labels=}')
    #   print(f'{torch.argmax(val_preds, dim=1)=}')

      val_f1 = f1_score(torch.argmax(val_labels,dim=1), torch.argmax(val_preds, dim=1), average='macro')
      if val_f1>max_val_f1:
        max_val_f1=val_f1
        torch.save(model.state_dict(), '/content/drive/MyDrive/Lessons/Models/multimodal_classification/' + 'Text_model_val_f1_{:.3f}_epoch{}.ckpt'.format(val_f1,epoch))
        best_model_wts = copy.deepcopy(model.state_dict())

  model.load_state_dict(best_model_wts)
  return(model,max_val_f1)


In [ ]:
model,max_val_f1=train_model(model,train_loader,val_loader,num_epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10, Validation F1 Score: 0.0000:   0%|          | 0/531 [00:00<?, ?it/s]

Epoch 2/10, Validation F1 Score: 0.6805:   0%|          | 0/531 [00:00<?, ?it/s]

Epoch 3/10, Validation F1 Score: 0.7032:   0%|          | 0/531 [00:00<?, ?it/s]

Epoch 4/10, Validation F1 Score: 0.7229:   0%|          | 0/531 [00:00<?, ?it/s]

Epoch 5/10, Validation F1 Score: 0.7272:   0%|          | 0/531 [00:00<?, ?it/s]

Epoch 6/10, Validation F1 Score: 0.7126:   0%|          | 0/531 [00:00<?, ?it/s]

Epoch 7/10, Validation F1 Score: 0.7269:   0%|          | 0/531 [00:00<?, ?it/s]

Epoch 8/10, Validation F1 Score: 0.7411:   0%|          | 0/531 [00:00<?, ?it/s]

Epoch 9/10, Validation F1 Score: 0.7338:   0%|          | 0/531 [00:00<?, ?it/s]

# Test part

In [ ]:
X_test = pd.read_csv(filepath+'X_test.csv')
X_test=X_test.drop(columns="Unnamed: 0")
X_test['designation'] = X_test['designation'].fillna('').apply(clean_text)
sequences = tokenizer.texts_to_sequences(X_test['designation'])

# Padding to max length of text
data = pad_sequences(sequences, maxlen=34)


In [ ]:
print(data)

In [ ]:
model = CNN_classifier(vocab_size, embedding_dim, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/Lessons/Models/multimodal_classification/Text_model_val_f1_0.734_epoch4.ckpt'))
model.to(device)
model.eval()

In [ ]:
val_preds = []
with torch.no_grad():
  for input in tqdm(data):
      input=torch.from_numpy(input)
      input=input.to(device)
      input=input.unsqueeze(0)
      val_outputs = model(input)[0]
      val_outputs=val_outputs.detach().cpu().numpy()
      val_preds.append(val_outputs)

In [ ]:
val_preds

In [ ]:
val_preds=np.argmax(val_preds,axis=1)

In [ ]:
val_preds=label_encoder.inverse_transform(val_preds)

In [ ]:
df_preds=pd.DataFrame(val_preds)
df_preds=df_preds.set_index(df_preds.index+84916)

In [ ]:
df_preds

In [ ]:
df_preds.to_csv("eval_text_designation.csv")